# Udacity's Vehicle detection project #
***
Project pipeline
---
* Feature extraction
    - color features
    - gradient features
* Train a classifier
* Sliding window search
* False positive detection
* Combined bounding boxes

In [ ]:
# import some useful packages
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.image as mpimg
import glob
import time
import random
import pickle
from skimage.feature import hog
from sklearn.preprocessing import StandardScaler
# NOTE: the next import is only valid for scikit-learn version <= 0.17
# for scikit-learn >= 0.18 use:
from sklearn.model_selection import train_test_split
# from sklearn.cross_validation import train_test_split
from scipy.ndimage.measurements import label
#from sklearn.svm import LinearSVC
from sklearn.svm import SVC

## Exploring data set and features

In [ ]:
# Explore data set, plot some different colour spaces


# Read in image
img = mpimg.imread('test_images/test1.jpg')

# Convert to different colour spaces
HSV = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
LUV = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
HLS = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
YUV = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
YCrCb = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)

# Plot
fig = plt.figure(figsize=(24,6))
plt.subplot(231)
plt.imshow(img, cmap='gray')
plt.title('Example Car Image')
plt.subplot(232)
plt.imshow(HSV, cmap='gray')
plt.title('HSV Visualization')
plt.subplot(233)
plt.imshow(LUV, cmap='gray')
plt.title('LUV Visualization')
plt.subplot(234)
plt.imshow(HLS, cmap='gray')
plt.title('HLS Visualization')
plt.subplot(235)
plt.imshow(YUV, cmap='gray')
plt.title('YUV Visualization')
plt.subplot(236)
plt.imshow(YCrCb, cmap='gray')
plt.title('YCrCb Visualization')


fig = plt.figure(figsize=(24,6))
plt.subplot(131)
plt.imshow(HLS[:,:,0])
plt.title('Y (YCrCb)')
plt.subplot(132)
plt.imshow(HLS[:,:,1])
plt.title('Cr (YCrCb)')
plt.subplot(133)
plt.imshow(HLS[:,:,2])
plt.title('Cb (YCrCb)')

img = mpimg.imread('test_images/test3.jpg')
# Convert to different colour spaces
HSV = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
LUV = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
HLS = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
YUV = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
YCrCb = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)

fig = plt.figure(figsize=(24,6))
plt.subplot(131)
plt.imshow(HLS[:,:,0])
plt.title('Y (YCrCb)')
plt.subplot(132)
plt.imshow(HLS[:,:,1])
plt.title('Cr (YCrCb)')
plt.subplot(133)
plt.imshow(HLS[:,:,2])
plt.title('Cb (YCrCb)')

# S in HSV
# U in LUV
# Cr in YCrCb
# U in YUV
# combined YCrCb or YUV

## Feature extraction

In [ ]:
# Define useful functions for Feature extraction

# Function to compute binned color features  
def bin_spatial(img, size=(32, 32)):
    features = cv2.resize(img, size).ravel() 
    # Return the feature vector
    return features

# Function to compute color histogram features  
def color_hist(img, nbins=32, bins_range=(0, 256)):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

# Function to return HOG features and visualization
def get_hog_features(img, orient, pix_per_cell, cell_per_block, vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True, 
                       visualise=vis, feature_vector=feature_vec)
        return features


In [ ]:
# Explore features for vehicles and non-vehicles

# Load image data
imagesveh = glob.glob('vehicles/**/*.png')
imagesNonveh = glob.glob('non-vehicles/**/*.png')
cars = []
notcars = []

for img in imagesveh:
        cars.append(img)
print(len(cars))
for img in imagesNonveh:
    notcars.append(img)     
print(len(notcars))

car_ind = np.random.randint(0, len(cars))
noncar_ind = np.random.randint(0,len(notcars))

# Read example image from data set
CarImage = mpimg.imread(cars[car_ind])
nonCarImage = mpimg.imread(notcars[noncar_ind])

# Plot the examples
fig = plt.figure()
plt.subplot(221)
plt.imshow(CarImage)
plt.title('Example Car Image')
plt.subplot(222)
plt.imshow(nonCarImage)
plt.title('Example Non Car Image')

# S in HSV
# U in LUV
# Cr in YCrCb
# U in YUV
# combined YCrCb or YUV

# Convert to different colour spaces
HSV = cv2.cvtColor(CarImage, cv2.COLOR_RGB2HSV)
LUV = cv2.cvtColor(CarImage, cv2.COLOR_RGB2LUV)
HLS = cv2.cvtColor(CarImage, cv2.COLOR_RGB2HLS)
YUV = cv2.cvtColor(CarImage, cv2.COLOR_RGB2YUV)
YCrCb_car = cv2.cvtColor(CarImage, cv2.COLOR_RGB2YCrCb)

YCrCb_nonCar = cv2.cvtColor(nonCarImage, cv2.COLOR_RGB2YCrCb)

############# HOG #############

#gray = cv2.cvtColor(CarImage, cv2.COLOR_RGB2GRAY)
# Define HOG parameters
orient = 11
pix_per_cell = 8
cell_per_block = 2
# Call our function with vis=True to see an image output
features, hog_image_carY = get_hog_features(YCrCb_car[:,:,0], orient, pix_per_cell, cell_per_block, vis=True, feature_vec=True)
features, hog_image_carCr = get_hog_features(YCrCb_car[:,:,1], orient, pix_per_cell, cell_per_block, vis=True, feature_vec=True)
features, hog_image_carCb = get_hog_features(YCrCb_car[:,:,2], orient, pix_per_cell, cell_per_block, vis=True, feature_vec=True)

#gray = cv2.cvtColor(nonCarImage, cv2.COLOR_RGB2GRAY)
features, hog_image_noncarY = get_hog_features(YCrCb_nonCar[:,:,0], orient, pix_per_cell, cell_per_block, vis=True, feature_vec=True)
features, hog_image_noncarCr = get_hog_features(YCrCb_nonCar[:,:,1], orient, pix_per_cell, cell_per_block, vis=True, feature_vec=True)
features, hog_image_noncarCb = get_hog_features(YCrCb_nonCar[:,:,2], orient, pix_per_cell, cell_per_block, vis=True, feature_vec=True)

# Plot the examples
fig = plt.figure(figsize=(12,3))
plt.subplot(231)
plt.imshow(hog_image_carY, cmap='gray')
plt.title('HOG Visualization Car Y')
plt.subplot(232)
plt.imshow(hog_image_carCr, cmap='gray')
plt.title('HOG Visualization Car Cr')
plt.subplot(233)
plt.imshow(hog_image_carY, cmap='gray')
plt.title('HOG Visualization Car Cb')
plt.subplot(234)
plt.imshow(hog_image_noncarY, cmap='gray')
plt.title('HOG Visualization non Car Y')
plt.subplot(235)
plt.imshow(hog_image_noncarCr, cmap='gray')
plt.title('HOG Visualization non Car Cr')
plt.subplot(236)
plt.imshow(hog_image_noncarY, cmap='gray')
plt.title('HOG Visualization non Car Cb')


############# Color histogram #############

feature_vec_car = color_hist(YCrCb_car, nbins=32, bins_range=(0, 256))
feature_vec_nonCar = color_hist(YCrCb_nonCar, nbins=32, bins_range=(0, 256))

'''
# Plot a figure with all bar charts
fig = plt.figure(figsize=(12,3))
plt.subplot(121)
plt.plot(feature_vec_car)
plt.xlim(0, 256)
plt.title('Histogram car')
plt.subplot(122)
plt.plot(feature_vec_nonCar)
plt.xlim(0, 256)
plt.title('Histogram non-car')
'''

############# Binned color #############    
feature_vec_car = bin_spatial(CarImage, size=(32, 32))
feature_vec_noncar = bin_spatial(nonCarImage, size=(32, 32))

# Plot features
fig = plt.figure(figsize=(12,3))
plt.subplot(121)
plt.plot(feature_vec_car)
plt.title('Spatially Binned Features car')
plt.subplot(122)
plt.plot(feature_vec_noncar)
plt.title('Spatially Binned Features non-car')


## Train Classifier

In [ ]:
# Define useful functions for Training classifier
# NOTE: Need to load functions for feature extraction as well
    
# Function to train classifier
# A function to extract features from a list of images 
# This function calls bin_spatial() and color_hist()
def extract_features(imgs, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel="ALL",
                        spatial_feat=True, hist_feat=True, hog_feat=True):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        file_features = []
        # Read in each one by one
        image = mpimg.imread(file)
        # apply color conversion if other than 'RGB'
        if color_space != 'RGB':
            if color_space == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif color_space == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
                # normalize
                feature_image[:,:,0] = 255/100*feature_image[:,:,0]
                feature_image[:,:,1] = 255/354*(feature_image[:,:,1]+134)
                feature_image[:,:,2] = 255/262*(feature_image[:,:,2]+140)
            elif color_space == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif color_space == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif color_space == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else: feature_image = np.copy(image)      

        if spatial_feat == True:
            spatial_features = bin_spatial(feature_image, size=spatial_size)
            file_features.append(spatial_features)
        if hist_feat == True:
            # Apply color_hist()
            hist_features = color_hist(feature_image, nbins=hist_bins)
            file_features.append(hist_features)
        if hog_feat == True:
        # Call get_hog_features() with vis=False, feature_vec=True
            if hog_channel == 'ALL':
                hog_features = []
                for channel in range(feature_image.shape[2]):
                    hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                        orient, pix_per_cell, cell_per_block, 
                                        vis=False, feature_vec=True))
                hog_features = np.ravel(hog_features)        
            else:
                hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                            pix_per_cell, cell_per_block, vis=False, feature_vec=True)
            # Append the new feature vector to the features list
            file_features.append(hog_features)
        features.append(np.concatenate(file_features))
    # Return list of feature vectors
    return features 

In [ ]:
# Read in vehicle and non-vehicle images
# Load images for training classifier

imagesveh = glob.glob('vehicles/**/*.png')
imagesNonveh = glob.glob('non-vehicles/**/*.png')
cars = []
notcars = []

i=0
for img in imagesveh:
    if 'image' in img:
        if i%5 == 0: # only add every 5th GTI data image
            cars.append(img)
        i+=1
    else:
        cars.append(img)
print(i)
print(len(cars))
for img in imagesNonveh:
    notcars.append(img)     
print(len(notcars))

In [ ]:
# Extract features and split into training and test set

# Reduce the sample size because HOG features are slow to compute 
sample_size = 6000
cars = random.sample(cars, sample_size)
notcars = random.sample(notcars, sample_size)

# TUNABLE PARAMETERS
color_space = 'YCrCb' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 11  # HOG orientations
pix_per_cell = 8 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
hog_channel = "ALL" # Can be 0, 1, 2, or "ALL"
spatial_size = (32,32) # Spatial binning dimensions
hist_bins = 32    # Number of histogram bins
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off

# S in HSV
# U in LUV
# Cr in YCrCb
# U in YUV
# combined YCrCb or YUV

# EXTRACT FEATURES
car_features = extract_features(cars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)
notcar_features = extract_features(notcars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)

X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

# Define the labels vector (1=car,0=notCar)
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.2, random_state=rand_state)

print('Using:',orient,'orientations',pix_per_cell,'pixels per cell and', cell_per_block,'cells per block')
print('Feature vector length:', len(X_train[0]))

In [ ]:
# Train classifier based on features

# Use a linear SVC with probability
#svc = LinearSVC()
svc = SVC(kernel='linear',probability=True)
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()

# Predict a subset
n_predict = 10
print('My SVC predicts: ', svc.predict(X_test[0:n_predict]))
print('For these',n_predict, 'labels: ', y_test[0:n_predict])
t2 = time.time()
print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')

In [ ]:
# When satisfied with result save classifier
dist_pickle = {"svc": svc, "scaler": X_scaler, "color_space": color_space, "orient": orient, "pix_per_cell": pix_per_cell, 
               "cell_per_block": cell_per_block, "hog_channel": hog_channel, "spatial_size": spatial_size,
               "hist_bins": hist_bins, "spatial_feat": spatial_feat, "hist_feat": hist_feat, "hog_feat": hog_feat}

pickle.dump(dist_pickle, open("svc_pickle.p", "wb" ))

## Search and classify new images

In [ ]:
# Define useful functions to Seach and classify new images


# Function to classify new image
# A function to extract features from a single image window
# This function is very similar to extract_features()
# just for a single image rather than list of images
def single_img_features(img, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel="ALL",
                        spatial_feat=True, hist_feat=True, hog_feat=True):    
    #1) Define an empty list to receive features
    img_features = []
    #2) Apply color conversion if other than 'RGB'
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
            # normalize
            feature_image[:,:,0] = 255/100*feature_image[:,:,0]
            feature_image[:,:,1] = 255/354*(feature_image[:,:,1]+134)
            feature_image[:,:,2] = 255/262*(feature_image[:,:,2]+140)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else: feature_image = np.copy(img)      
    #3) Compute spatial features if flag is set
    if spatial_feat == True:
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        #4) Append features to list
        img_features.append(spatial_features)
    #5) Compute histogram features if flag is set
    if hist_feat == True:
        hist_features = color_hist(feature_image, nbins=hist_bins)
        #6) Append features to list
        img_features.append(hist_features)
    #7) Compute HOG features if flag is set
    if hog_feat == True:
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.extend(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))      
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        #8) Append features to list
        img_features.append(hog_features)

    #9) Return concatenated array of features
    return np.concatenate(img_features)

# Function to draw bounding boxes
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return imcopy
        
# A function that takes an image, start and stop positions in both x and y, 
# window size (x and y dimensions) and overlap fraction (for both x and y)
# and defines image boxes to classify as vehicles/non-vehicles
def slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None], 
                    xy_window=(64, 64), xy_overlap=(0.5, 0.5)):
    # If x and/or y start/stop positions not defined, set to image size
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]
    # Compute the span of the region to be searched    
    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]
    # Compute the number of pixels per step in x/y
    nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))
    # Compute the number of windows in x/y
    nx_buffer = np.int(xy_window[0]*(xy_overlap[0]))
    ny_buffer = np.int(xy_window[1]*(xy_overlap[1]))
    nx_windows = np.int((xspan-nx_buffer)/nx_pix_per_step) 
    ny_windows = np.int((yspan-ny_buffer)/ny_pix_per_step) 
    # Initialize a list to append window positions to
    window_list = []
    # Loop through finding x and y window positions
    # Note: you could vectorize this step, but in practice
    # you'll be considering windows one by one with your
    # classifier, so looping makes sense
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            # Calculate window position
            startx = xs*nx_pix_per_step + x_start_stop[0]
            endx = startx + xy_window[0]
            starty = ys*ny_pix_per_step + y_start_stop[0]
            endy = starty + xy_window[1]
            # Append window position to list
            window_list.append(((startx, starty), (endx, endy)))
    # Return the list of windows
    return window_list

# A function used for searching an image for vehicles. 
# The windows to be searched is the output of slide_windows()
def search_windows(img, windows, clf, scaler, color_space='RGB', 
                    spatial_size=(32, 32), hist_bins=32, 
                    hist_range=(0, 256), orient=9, 
                    pix_per_cell=8, cell_per_block=2, 
                    hog_channel="ALL", spatial_feat=True, 
                    hist_feat=True, hog_feat=True):

    #1) Create an empty list to receive positive detection windows
    on_windows = []
    #2) Iterate over all windows in the list
    for window in windows:
        #3) Extract the test window from original image
        test_img = cv2.resize(img[window[0][1]:window[1][1], window[0][0]:window[1][0]], (64, 64))      
        #4) Extract features for that window using single_img_features()
        features = single_img_features(test_img, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, 
                            hog_channel=hog_channel, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat)
        #5) Scale extracted features to be fed to classifier
        test_features = scaler.transform(np.array(features).reshape(1, -1))
        #6) Predict using classifier
        #prediction = clf.predict(test_features)
        prediction = clf.predict_proba(test_features)
                   
        #7) If certain (>70%) then save the window
        if prediction[0,1] >= 0.70:
            on_windows.append(window)
            #print(prediction[0,1])
    #8) Return windows for positive detections
    return on_windows

In [ ]:
# Define windows to search for vehicles in, save outcome (only needed once)

# Read in image
image = mpimg.imread('test_images/test1.jpg')

# Define windows to search in
y_max = image.shape[0] 
y_min = y_max//2 
x_max = image.shape[1]

# size 1
box_lim = y_max//3
des_overlap = 0.75
n_ish = (x_max-box_lim)/box_lim/(1-des_overlap)+1
n = round(n_ish)
print(n)
overlap = 1-(x_max-box_lim)/box_lim/(n-1)

windows = slide_window(image, x_start_stop=[None, None], y_start_stop=[y_min, y_max-50], 
                    xy_window=(box_lim, box_lim), xy_overlap=(overlap,0.9))                
# size 2
box_lim = y_max//4
des_overlap = 0.7
n_ish = (x_max-box_lim)/box_lim/(1-des_overlap)+1
n = round(n_ish)
print(n)
overlap = 1-(x_max-box_lim)/box_lim/(n-1)

windows2 = slide_window(image, x_start_stop=[None, None], y_start_stop=[y_min, y_max-100], 
                    xy_window=(box_lim, box_lim), xy_overlap=(overlap,overlap))                    
# size 3
box_lim = y_max//5
des_overlap = 0.70 
n_ish = (x_max-box_lim)/box_lim/(1-des_overlap)+1
n = round(n_ish) 
print(n)
overlap = 1-(x_max-box_lim)/box_lim/(n-1) 

windows3 = slide_window(image, x_start_stop=[275, None], y_start_stop=[y_min, y_max-y_max//4], 
                    xy_window=(box_lim, box_lim), xy_overlap=(overlap,0.80))
# size 4
box_lim = y_max//8
des_overlap = 0.75
n_ish = (x_max-box_lim)/box_lim/(1-des_overlap)+1
n = round(n_ish)
print(n)
overlap = 1-(x_max-box_lim)/box_lim/(n-1)
print(box_lim)

windows4 = slide_window(image, x_start_stop=[310, None], y_start_stop=[y_min+30, y_max-y_max//4], 
                    xy_window=(box_lim, box_lim), xy_overlap=(overlap,overlap))
# size 5
box_lim = y_max//6
des_overlap = 0.75
n_ish = (x_max-box_lim)/box_lim/(1-des_overlap)+1
n = round(n_ish)
print(n)
overlap = 1-(x_max-box_lim)/box_lim/(n-1)
print(box_lim)

windows5 = slide_window(image, x_start_stop=[360, None], y_start_stop=[y_min, y_max-y_max//4], 
                    xy_window=(box_lim, box_lim), xy_overlap=(overlap,overlap))
# size 6
box_lim = y_max//9
des_overlap = 0.75#0.75 
n_ish = (x_max-box_lim)/box_lim/(1-des_overlap)+1
n = round(n_ish)
print(n)
overlap = 1-(x_max-box_lim)/box_lim/(n-1)
print(box_lim)

windows6 = slide_window(image, x_start_stop=[360, 1100], y_start_stop=[y_min, 500], 
                    xy_window=(box_lim, box_lim), xy_overlap=(overlap,overlap))

# add all windows together
windows.extend(windows2+windows3+windows4+windows5+windows6) 

# Plot to see how it looks in image
window_img = draw_boxes(image, windows, color=(0, 0, 255), thick=6)                    
plt.imshow(window_img)

# save searching window
pickle.dump(windows, open("windows_pickle.p", "wb" ))

In [ ]:
# Load saved classifier and search windows

# Use trained classifier to find vehicles in new images
dist_pickle = pickle.load( open("svc_pickle.p", "rb" ) )
svc = dist_pickle["svc"]
X_scaler = dist_pickle["scaler"]
color_space = dist_pickle["color_space"]
orient = dist_pickle["orient"]
pix_per_cell = dist_pickle["pix_per_cell"]
cell_per_block = dist_pickle["cell_per_block"]
hog_channel = dist_pickle["hog_channel"]
spatial_size = dist_pickle["spatial_size"]
hist_bins = dist_pickle["hist_bins"]
spatial_feat = dist_pickle["spatial_feat"]
hist_feat = dist_pickle["hist_feat"]
hog_feat = dist_pickle["hog_feat"]

# Load windows to search in
windows = pickle.load( open("windows_pickle.p", "rb" ) )
print(np.shape(windows))

In [ ]:
# Search for vehicles in images

# Load image to classify
image = mpimg.imread('videoFrames/frame943.png')

# create image copy for plotting
draw_image = np.copy(image)

# Uncomment the following line if you extracted training
# data from .png images (scaled 0 to 1 by mpimg) and the
# image you are searching is a .jpg (scaled 0 to 255)
#image = image.astype(np.float32)/255

# Look for vehicles in defined windows
hot_windows = search_windows(image, windows, svc, X_scaler, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)                       

# draw windows found as vehicles, plot
window_img = draw_boxes(draw_image, hot_windows, color=(0, 0, 255), thick=6)                    
plt.imshow(window_img)


## Remove false positives and combine boxes

In [ ]:
# Define useful functions for Removing false positives and define bounding boxes

# Function to compute the heatmap depending on found 
# windows of vehicles
def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes  
    for frame in bbox_list:
        for box in frame:
            # Add += 1 for all pixels inside each bbox
            # Assuming each "box" takes the form ((x1, y1), (x2, y2))
            heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1

    # Return updated heatmap
    return heatmap

# Function to filter false positives
def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap

# Function to draw combined boxes
def draw_labeled_bboxes(img, labels):
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # Draw the box on the image
        cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)
    # Return the image
    return img

In [ ]:
# Get rid of false positives and combine true positives
# NOTE: based on output from "Search and classify new images"
box_list = []
box_list.append(hot_windows)

# Add heat to each box in box list
heat = np.zeros_like(image[:,:,0]).astype(np.float)
heat = add_heat(heat,box_list)
    
# Apply threshold to help remove false positives
heat = apply_threshold(heat,3)

# Visualize the heatmap when displaying    
heatmap = np.clip(heat, 0, 255)

# Find final boxes from heatmap using label function
labels = label(heatmap)
draw_img = draw_labeled_bboxes(np.copy(image), labels)

fig = plt.figure()
plt.subplot(121)
plt.imshow(draw_img)
plt.title('Bounded')
plt.subplot(122)
plt.imshow(heatmap, cmap='hot')
plt.title('Heat Map')
fig.tight_layout()


# Video pipeline
***
A video can be processed by loading the following sections. Note that a classifier ("svc_pickle.p") and windows to search for vehicles in ("windows_pickle.p") is needed as inputs. 

In [ ]:
# import needed packages
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import cv2
import glob
import time
import pickle
import random
from scipy.ndimage.measurements import label
#from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
# NOTE: the next import is only valid for scikit-learn version <= 0.17
# for scikit-learn >= 0.18 use:
from sklearn.model_selection import train_test_split
#from sklearn.cross_validation import train_test_split


In [ ]:
# Define useful functions

########### Extracting features and classify functions ##########

# Function to compute binned color features  
def bin_spatial(img, size=(32, 32)): 
    features = cv2.resize(img, size).ravel() 
    # Return the feature vector
    return features

# Function to compute color histogram features  
def color_hist(img, nbins=32, bins_range=(0, 256)): 
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

# Function to return HOG features and visualization
def get_hog_features(img, orient, pix_per_cell, cell_per_block, vis=False, feature_vec=True):  
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True, 
                       visualise=vis, feature_vector=feature_vec)
        return features

# Function to classify new image
# A function to extract features from a single image window
# This function is very similar to extract_features()
# just for a single image rather than list of images
def single_img_features(img, color_space='RGB', spatial_size=(32, 32), 
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel="ALL",
                        spatial_feat=True, hist_feat=True, hog_feat=True):    
    #1) Define an empty list to receive features
    img_features = []
    #2) Apply color conversion if other than 'RGB'
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
            # normalize
            feature_image[:,:,0] = 255/100*feature_image[:,:,0]
            feature_image[:,:,1] = 255/354*(feature_image[:,:,1]+134)
            feature_image[:,:,2] = 255/262*(feature_image[:,:,2]+140)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else: feature_image = np.copy(img)      
    #3) Compute spatial features if flag is set
    if spatial_feat == True:
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        #4) Append features to list
        img_features.append(spatial_features)
    #5) Compute histogram features if flag is set
    if hist_feat == True:
        hist_features = color_hist(feature_image, nbins=hist_bins)
        #6) Append features to list
        img_features.append(hist_features)
    #7) Compute HOG features if flag is set
    if hog_feat == True:
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.extend(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))      
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        #8) Append features to list
        img_features.append(hog_features)

    #9) Return concatenated array of features
    return np.concatenate(img_features)

# A function used for searching an image for vehicles. 
# The windows to be searched is the output of slide_windows()
def search_windows(img, windows, clf, scaler, color_space='RGB', 
                    spatial_size=(32, 32), hist_bins=32, 
                    hist_range=(0, 256), orient=9, 
                    pix_per_cell=8, cell_per_block=2, 
                    hog_channel="ALL", spatial_feat=True, 
                    hist_feat=True, hog_feat=True):

    #1) Create an empty list to receive positive detection windows
    on_windows = []
    #2) Iterate over all windows in the list
    for window in windows:
        #3) Extract the test window from original image
        test_img = cv2.resize(img[window[0][1]:window[1][1], window[0][0]:window[1][0]], (64, 64))      
        #4) Extract features for that window using single_img_features()
        features = single_img_features(test_img, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, 
                            hog_channel=hog_channel, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat)
        #5) Scale extracted features to be fed to classifier
        test_features = scaler.transform(np.array(features).reshape(1, -1))
        #6) Predict using classifier
        #prediction = clf.predict(test_features)
        prediction = clf.predict_proba(test_features)
                   
        #7) If certain (>=70%) then save the window
        if prediction[0,1] >= 0.70:
            on_windows.append(window)
            #print(prediction[0,1])
    #8) Return windows for positive detections
    return on_windows


########### False positives functions ############

# Function to compute the heatmap depending on found 
# windows of vehicles
def add_heat(heatmap, bbox_list): 
    # Iterate through list of bboxes  
    for frame in bbox_list:
        for box in frame:
            # Add += 1 for all pixels inside each bbox
            # Assuming each "box" takes the form ((x1, y1), (x2, y2))
            heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1

    # Return updated heatmap
    return heatmap

# Function to filter false positives
def apply_threshold(heatmap, threshold): 
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap

# Function to draw combined boxes
def draw_labeled_bboxes(img, labels): 
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # Draw the box on the image
        cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)
    # Return the image
    return img

In [ ]:
# Function used for processing image

def process_image(img):
    # Load data for classifier
    # Use trained classifier to find vehicles in new images
    dist_pickle = pickle.load( open("svc_pickle.p", "rb" ) )
    svc = dist_pickle["svc"]
    X_scaler = dist_pickle["scaler"]
    color_space = dist_pickle["color_space"]
    orient = dist_pickle["orient"]
    pix_per_cell = dist_pickle["pix_per_cell"]
    cell_per_block = dist_pickle["cell_per_block"]
    hog_channel = dist_pickle["hog_channel"]
    spatial_size = dist_pickle["spatial_size"]
    hist_bins = dist_pickle["hist_bins"]
    spatial_feat = dist_pickle["spatial_feat"]
    hist_feat = dist_pickle["hist_feat"]
    hog_feat = dist_pickle["hog_feat"]

    # Load data defining searching areas
    windows = pickle.load(open("windows_pickle.p", "rb" ))
   
    # if image > 1 it's read as BGR [0,255] for video stream, 
    # we have used matplotlib and PNG images -> RGB & values [0,1]
    if np.amax(img) > 1:
        img_cpy = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_cpy = img_cpy.astype(np.float32)/255
    else:
        img_cpy = np.copy(img)

    # Uncomment the following line if you extracted training
    # data from .png images (scaled 0 to 1 by mpimg) and the
    # image you are searching is a .jpg (scaled 0 to 255)
    #img = img.astype(np.float32)/255
    
    # Look for vehicles in defined windows
    hot_windows = search_windows(img_cpy, windows, svc, X_scaler, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat) 
    
    # add new windows to history
    vehicle_history.detected_windows.append(hot_windows)
    
    # Add heat to each windows in last (up to 5) frames hot_windows
    heat = np.zeros_like(img[:,:,0]).astype(np.float)
    heat = add_heat(heat,vehicle_history.detected_windows)
    
    # use heat maps over several frames after 5 frames detected
    if len(vehicle_history.detected_windows)==5: #values since before
        #vehicle_history.detected = True
        # Apply threshold to help remove false positives
        heat = apply_threshold(heat,2)
        # Throw away first value (old data)
        vehicle_history.detected_windows.pop(0)
                                                
    # Visualize the heatmap when displaying    
    heatmap = np.clip(heat, 0, 255)

    # Find final boxes from heatmap using label function
    labels = label(heatmap)
    out_img = draw_labeled_bboxes(np.copy(img), labels)    
   
    return out_img

In [ ]:
# Define vehicle information class to keep track of history
class Vehicle():
    def __init__(self):
        self.detected = False
        self.detected_windows = [] # windows detected as vehicles in last nth images

In [ ]:
# Save video as images to be able to analyse just some images
from moviepy.editor import VideoFileClip

clip1 = VideoFileClip("project_video.mp4")
i = 0
for frame in clip1.iter_frames():
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    cv2.imwrite("videoFrames/frame%d.png" % i, img)     # save frame as PNG file
    i += 1

In [ ]:
# Process separate image (Note, will not use heat thresholding)

#image = mpimg.imread('test_images/test6.jpg')
image = mpimg.imread('videoFrames/frame633.png')

# Uncomment the following line if the image you 
# are searching is a .jpg (scaled 0 to 255)
#image = image.astype(np.float32)/255

# Define a vehicle
vehicle_history = Vehicle()

t = time.time()
out_img = process_image(image)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to predict image...')
fig = plt.figure()
plt.imshow(out_img)

In [ ]:
# Process sequence of images, will be the same as video

# Define a vehicle
vehicle_history = Vehicle()

for idx in range(937,997): # values between 0 and 1260
    # Read image
    image = mpimg.imread("videoFrames/frame" + str(idx) + ".png") 
    # Process image
    out_img = process_image(image)                  

    # Save image
    mpimg.imsave("OutvideoFrames/frame" + str(idx) + "-out.png", out_img)

In [ ]:
# write sequence of images to video
from moviepy.editor import ImageSequenceClip

# Save image names in list
videoImg = []

for idx in range(0,1260):
    videoImg.append("OutvideoFrames/frame" + str(idx) + "-out.png")     

# Create video
video_file = 'project_video_edit_finalTest.mp4'
clip = ImageSequenceClip(videoImg, fps=25)
clip.write_videofile(video_file, audio=False)

In [ ]:
# Detect vehicles in video stream

# Define a vehicle
#vehicle_history = Vehicle()

# Process video
#v_output = 'test_video_edit.mp4'
#clip1 = VideoFileClip("test_video.mp4")
#out_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
#%time out_clip.write_videofile(v_output, audio=False)